<a href="https://colab.research.google.com/github/PyBeginner1/CriticalHeatFluxPrediction/blob/main/CriticalHeatFluxPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestRegressor

In [24]:
data = pd.read_csv('/content/cat.csv')

In [25]:
data

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,1,Inasaka,tube,0.39,5600,-0.1041,3.0,3.0,100,11.3
1,2,Inasaka,tube,0.31,6700,-0.0596,3.0,3.0,100,10.6
2,3,Inasaka,tube,0.33,4300,-0.0395,3.0,3.0,100,7.3
3,4,Inasaka,tube,0.62,6400,-0.1460,3.0,3.0,100,12.8
4,5,Inasaka,tube,0.64,4700,-0.0849,3.0,3.0,100,11.0
...,...,...,...,...,...,...,...,...,...,...
1860,1861,Richenderfer,plate,1.01,1500,-0.0218,15.0,120.0,10,9.4
1861,1862,Richenderfer,plate,1.01,1500,-0.0434,15.0,120.0,10,10.4
1862,1863,Richenderfer,plate,1.01,2000,-0.0109,15.0,120.0,10,10.8
1863,1864,Richenderfer,plate,1.01,2000,-0.0218,15.0,120.0,10,10.9


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1865 entries, 0 to 1864
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   1865 non-null   int64  
 1   author               1865 non-null   object 
 2   geometry             1865 non-null   object 
 3   pressure [MPa]       1865 non-null   float64
 4   mass_flux [kg/m2-s]  1865 non-null   int64  
 5   x_e_out [-]          1865 non-null   float64
 6   D_e [mm]             1865 non-null   float64
 7   D_h [mm]             1865 non-null   float64
 8   length [mm]          1865 non-null   int64  
 9   chf_exp [MW/m2]      1865 non-null   float64
dtypes: float64(5), int64(3), object(2)
memory usage: 145.8+ KB


In [27]:
data.isnull().sum()

id                     0
author                 0
geometry               0
pressure [MPa]         0
mass_flux [kg/m2-s]    0
x_e_out [-]            0
D_e [mm]               0
D_h [mm]               0
length [mm]            0
chf_exp [MW/m2]        0
dtype: int64

In [28]:
data.describe()

,id,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
count,1865.000000,1865.000000,1865.000000,1865.000000,1865.000000,1865.000000,1865.000000,1865.000000
mean,933.000000,10.010949,2862.647721,0.016179,9.417212,16.167721,911.340483,3.854638
std,538.523444,4.282715,1656.412247,0.117575,6.333807,21.182870,726.718974,1.985535
min,1.000000,0.100000,0.000000,-0.866700,1.000000,1.000000,10.000000,0.800000
25%,467.000000,6.890000,1519.000000,-0.048300,5.600000,5.600000,432.000000,2.400000
50%,933.000000,10.340000,2590.000000,0.024400,8.500000,10.300000,625.000000,3.500000
75%,1399.000000,13.790000,3933.000000,0.100600,11.100000,15.200000,1778.000000,4.800000
max,1865.000000,20.680000,7975.000000,0.232000,37.500000,120.000000,3048.000000,19.300000


In [29]:
data = data.drop(['id', 'author'],axis =1)

Shuffle the dataset since kfold doesnt

In [37]:
data = data.sample(frac = 1.0, random_state = 1)

In [38]:
data

,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
400,tube,6.89,1967,0.1203,12.8,12.8,1930,2.8
571,tube,11.03,2034,0.0616,9.3,9.3,762,3.2
351,tube,7.00,4069,0.0636,10.8,10.8,864,3.2
1479,annulus,13.79,2729,0.0537,5.6,15.2,2134,1.7
1371,tube,13.79,690,-0.4854,11.1,11.1,457,3.6
...,...,...,...,...,...,...,...,...
905,tube,13.79,4910,0.1131,4.7,4.7,318,3.0
1791,annulus,6.85,2292,0.0330,8.5,24.6,1778,4.0
1096,tube,13.79,2116,0.0926,7.8,7.8,591,2.1
235,tube,6.89,4042,-0.0008,10.3,10.3,762,4.0


In [39]:
X = data.drop('chf_exp [MW/m2]', axis =1)
Y = data['chf_exp [MW/m2]']

In [40]:
X

,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm]
400,tube,6.89,1967,0.1203,12.8,12.8,1930
571,tube,11.03,2034,0.0616,9.3,9.3,762
351,tube,7.00,4069,0.0636,10.8,10.8,864
1479,annulus,13.79,2729,0.0537,5.6,15.2,2134
1371,tube,13.79,690,-0.4854,11.1,11.1,457
...,...,...,...,...,...,...,...
905,tube,13.79,4910,0.1131,4.7,4.7,318
1791,annulus,6.85,2292,0.0330,8.5,24.6,1778
1096,tube,13.79,2116,0.0926,7.8,7.8,591
235,tube,6.89,4042,-0.0008,10.3,10.3,762


In [41]:
Y

400     2.8
571     3.2
351     3.2
1479    1.7
1371    3.6
       ... 
905     3.0
1791    4.0
1096    2.1
235     4.0
1061    2.0
Name: chf_exp [MW/m2], Length: 1865, dtype: float64

In [42]:
X['geometry'].value_counts()

tube       1439
annulus     378
plate        48
Name: geometry, dtype: int64

In [46]:
def build_model():

  #onehot encode geometry column
  nominal_transformer = Pipeline(steps = [
                    ('onehot', OneHotEncoder(sparse = False, handle_unknown = 'ignore'))
  ])

  #for column geometry
  preprocessing = ColumnTransformer(transformers = [('nominal', nominal_transformer, ['geometry'])], remainder = 'passthrough')



  model = Pipeline(steps = [
                            ('preprocessor', preprocessing),
                            ('regressor', RandomForestRegressor(random_state = 1))
  ])

  return model

Train

In [51]:
kf = KFold(n_splits = 5)

rmses = []

for train_idx, test_idx in kf.split(X):
  X_train = X.iloc[train_idx, :]
  X_test = X.iloc[test_idx, :]
  Y_train = Y.iloc[train_idx]
  Y_test = Y.iloc[test_idx]

  model = build_model()
  model.fit(X_train, Y_train)

  y_pred = model.predict(X_test)

  rmse = np.sqrt(np.mean((Y_test - y_pred)**2))

  rmses.append(rmse)

final_rmse = np.mean(rmses)

print('RMSE:{:.2f}'.format(final_rmse))


RMSE:0.64
